In [7]:
import importlib
import data
from gurobipy import *

# get data from excel
dataframes = data.get_all_pandas()

In [8]:
##### Respect du nombre de voies de chantier #####
def retrieve_occupations(OCCUPATIONS):
    # print(len(OCCUPATIONS))
    # print(OCCUPATIONS[0])
    # print(type(OCCUPATIONS[0]))
    # print(OCCUPATIONS[0].getValue())
    chantiers_df = dataframes["chantiers_df"]
    L = {}
    occupation_limits = [chantiers_df[chantiers_df["Chantier"]=="WPY_REC"]["Nombre de voies"].iloc[0],
            chantiers_df[chantiers_df["Chantier"]=="WPY_FOR"]["Nombre de voies"].iloc[0],
            chantiers_df[chantiers_df["Chantier"]=="WPY_DEP"]["Nombre de voies"].iloc[0]]
    for tache_name, sillon_id, line_expr in OCCUPATIONS["DEB"]:
        v = line_expr.getValue()
        
        # if v == occupation_limits[0]:
        #     print("DEB limite atteinte")
        if v > occupation_limits[0]:
            print("DEB limite DEPASSEE")
            
        L[tache_name, sillon_id] = v
        
    for tache_name, sillon_id, line_expr in OCCUPATIONS["FOR"]:
        v = line_expr.getValue()
        # if v == occupation_limits[1]:
        #     print("FOR limite atteinte")
        if v > occupation_limits[1]:
            print("FOR limite DEPASSEE")
        L[tache_name, sillon_id] = v


    for tache_name, sillon_id, line_expr in OCCUPATIONS["DEG"]:
        v = line_expr.getValue()
        # if v == occupation_limits[2]:
        #     print("DEG limite atteinte")
        if v > occupation_limits[2]:
            print("DEG limite DEPASSEE")
        L[tache_name, sillon_id] = v
    return L,occupation_limits


In [9]:
# Excel builder
from openpyxl import Workbook
from datetime import datetime
from utils import min_to_jour
import utils

def show_solution(model, debug=False):
    # print(var_dict)
    print("Objectif minimal: ", m.objVal)
    #build excel of tasks
    workbook = Workbook()
    sheet = workbook.active
    sillons_df = dataframes["sillons_df"]
    taches_df = dataframes["taches_df"]
    machines_df = dataframes["machines_df"]
    L, occupation_limits = retrieve_occupations(OCCUPATIONS)
    occupation_keys = L.keys()

    sheet["A1"] = "Id_tache"
    sheet["B1"] = "Type de tache"
    sheet["C1"] = "Jour"
    sheet["D1"] = "HDébut"
    sheet["E1"] = "Durée"
    sheet["F1"] = "Sillon"
    sheet["G1"] = "Minute"
    sheet["H1"] = "Occupation"
    sheet["I1"] = "Journée de service RAJC"
    row = 2
    for tache in var_dict.keys():
        for sillon_id in var_dict[tache].keys():
            try:
                
                
                sheet[f"B{row}"] = tache
                sheet[f"A{row}"] = tache + "_" + sillon_id
                sheet[f"C{row}"] = min_to_jour(var_dict[tache][sillon_id].X)[0]
                sheet[f"D{row}"] = min_to_jour(var_dict[tache][sillon_id].X)[1]
                if tache in list(machines_df["Machine"]):
                    sheet[f"E{row}"] = machines_df[machines_df["Machine"] == tache]["Duree "].iloc[0]
                elif tache in list(taches_df["Type de tache humaine"]): 
                    sheet[f"E{row}"] = taches_df[taches_df["Type de tache humaine"] == tache]["Durée"].iloc[0]
                sheet[f"F{row}"] = sillon_id
                sheet[f"G{row}"] = var_dict[tache][sillon_id].X
                if (tache, sillon_id) in occupation_keys:
                    sheet[f"H{row}"] = L[tache, sillon_id]
                if tache not in list(machines_df["Machine"]):
                    duree = int(taches_df[taches_df["Type de tache humaine"]==tache]["Durée"].iloc[0]) 
                    test_unique_jds = 0
                    for tache_key, sillon_id_key, r,a,j,c in links_dict.keys():
                        if tache_key==tache and sillon_id_key==sillon_id and links_dict[tache_key, sillon_id_key, r,a,j,c].X == 1:
                            if test_unique_jds>=1:
                                print("ERROR ON J2.C1", test_unique_jds, tache, sillon_id, r,a,j,c )
                            test_unique_jds += 1
                            jds_tsring = f"{r}, {a}, {j}, {c}"
                            # check JDS is correct
                            if True:
                                min_start, min_end = utils.get_min_from_rajc(r,j,c, dataframes["roulements_df"])
                                # print(min_start, var_dict[tache][sillon_id].X, min_end)
                                # print(duree)
                                assert min_start <= var_dict[tache][sillon_id].X and var_dict[tache][sillon_id].X + duree <= min_end , f"JDS: {r,a,j,c},min_start: {min_start}, task: {var_dict[tache][sillon_id].X}, min_end: {min_end}"
                try:
                    sheet[f"I{row}"] = jds_tsring + f"INFO : minute start : {min_start}, minute end : {min_end}"
                except:
                    raise Exception("JDS not found, ",tache, sillon_id)
                row += 1
            except:
                if not debug:
                    raise Exception(f"Error on task : {tache}, sillon {sillon_id}")
    
    ##ocupation
    
    occupation_sheet = workbook.create_sheet("Occupation")
    sheet = occupation_sheet
    sheet["A1"] = "Mesure"
    sheet["B1"] = "WPY_REC"
    sheet["C1"] = "WPY_FOR"
    sheet["D1"] = "WPY_DEP"
    
    
    sheet["A2"] = "Taux max occupation"
    sheet["A3"] = "Nb max occupation"
    sheet["A4"] = "Nb max par chantier"
    
    chantier = "WPY_REC"
    occupation_max = max([L["arrivée Reception", sillon_id_key] for tache_name_key, sillon_id_key in L.keys() if tache_name_key == "arrivée Reception" ])
    capacity = occupation_limits[0]
    sheet["B2"] = 100*occupation_max/capacity
    sheet["B3"] = occupation_max
    sheet["B4"] = capacity
    
    chantier = "WPY_FOR"
    occupation_max = max([L["débranchement", sillon_id_key] for tache_name_key, sillon_id_key in L.keys() if tache_name_key == "débranchement" ])
    capacity = occupation_limits[1]
    sheet["C2"] = 100*occupation_max/capacity
    sheet["C3"] = occupation_max
    sheet["C4"] = capacity
    
    chantier = "WPY_DEP"
    
    occupation_max =max([L["dégarage / bouger de rame", sillon_id_key] for tache_name_key, sillon_id_key in L.keys() if tache_name_key == "dégarage / bouger de rame" ])
    capacity = occupation_limits[2]
    sheet["D2"] = 100*occupation_max/capacity
    sheet["D3"] = occupation_max
    sheet["D4"] = capacity

    
    ##jds 
    
    jds_sheet = workbook.create_sheet("JDS")
    sheet = jds_sheet
    sheet["A1"] = "jds"
    sheet["B1"] = "Used ?"
    idx=1
    for (roulement,a,jour,c) in all_active_jds.keys():
        sheet[f"A{idx}"] = roulement,a,jour,c
        sheet[f"B{idx}"] = all_active_jds[roulement,a,jour,c].X
        idx += 1
    
    currentDateAndTime = datetime.now().strftime("%d-%m_%H-%M-%S")
    workbook.save(filename=f"results/{data.NAME}-{currentDateAndTime}.xlsx")
# show_solution(m)


In [10]:
# imports
import variables
import contraintes
import contraintesJ2
import importlib
import utils
import objectif
importlib.reload(variables) #to remive in final .py

importlib.reload(utils)

<module 'utils' from 'c:\\Users\\remgo\\Desktop\\ProjetST7\\git\\utils.py'>

=================================================

In [13]:
# GO
#to remive in final .py
importlib.reload(utils)
importlib.reload(variables)
importlib.reload(contraintes) 
importlib.reload(contraintesJ2)
importlib.reload(objectif)
# create model
m = Model()

# generate variables (integers)
var_dict = variables.generate_variablesJ1(m, dataframes)
links_dict = variables.generate_variablesJ2(m,var_dict,dataframes)
# generate variables
B, OCCUPATIONS, max_horizon = contraintes.generate_contraintes(m, dataframes, var_dict)
contraintesJ2.generate_contraintes(m, dataframes, var_dict, links_dict)

# final_train_task = dataframes["sillons_df"].copy()
# final_train_task = final_train_task[final_train_task["LDEP"]=="WPY"]
# final_train_task = final_train_task[final_train_task["JARR"]==final_train_task["JARR"].max()]
# # final_train_task = final_train_task[final_train_task["HDEP"]==final_train_task["HDEP"].max()]["train_id"].iloc[0]
# # final_train_task = var_dict["essai de frein départ"][final_train_task]
# final_train_task = quicksum([var_dict["essai de frein départ"][temp] for temp in list(final_train_task["train_id"])])
# print(final_train_task)

all_active_jds, objectif_expr = objectif.get_objectif(m, var_dict, links_dict, dataframes)
m.setObjective(objectif_expr, GRB.MINIMIZE)
# autre idée : maximiser la moyenne de l'ecart entre frein de depart et HDEP
# aute idee minimiser occupation
# m.printStats()
m.update()
m.Params.preSparsify = 2
m.optimize()


Initialising variables:   0%|          | 0/217 [00:00<?, ?it/s]

Link variables:   0%|          | 0/5 [00:00<?, ?it/s]

BUILDING CONSTRAINTS FOR J1:   0%|          | 0/8 [00:00<?, ?it/s]

BUILDING CONSTRAINTS FOR J2:   0%|          | 0/4 [00:00<?, ?it/s]

249800
Starting C4


  0%|          | 0/124900 [00:00<?, ?it/s]

DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulement_depart 1 1 1
DEBUG roulemen

Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

In [ ]:
show_solution(m, debug=False)

Objectif minimal:  12645.0
